In [ ]:
import itertools
import pandas as pd
import numpy as np
import plotly.express as px
import skimage
import scipy
import plotly.io as pio
# pio.renderers.default = "jupyterlab"
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN


def bump(x):
    return np.where(np.abs(x) < 1, np.exp(1 / (x ** 2 - 1) + 1) ** 0.2, 0)

In [ ]:
theory_file = r"D:\DATA\4ranges-volume-momspe-2d-sw.dat"
d = []
for line in open(theory_file):
    if len(line.strip().split("  ")) == 6:
        d.append([float(x) for x in line.strip().split('  ')])

d = np.asarray(d)
d = d[d[:, 0] < 0.35]


def fit_fn(theta, theta0, a, b):
    return a * np.cos((theta - theta0) * 2) + b


pr = d[:, 0]
theta = d[:, 1] % (2 * np.pi) - np.pi
I = d[:, 4]

grid_size = (len(set(pr)), len(set(theta)))
pr = pr.reshape(grid_size)
r_v = np.asarray(sorted(np.unique(pr)))
theta = theta.reshape(grid_size)
theta_v = np.asarray(sorted(np.unique(theta)))
I = I.reshape(grid_size)

r = np.linspace(0, len(r_v), 1024)
theta = np.linspace(0, len(theta_v), 1024)

rr, tt = np.meshgrid(r, theta)
I = scipy.ndimage.map_coordinates(I, [rr.flatten(), tt.flatten()])

r_v = np.linspace(min(r_v), max(r_v), 1024)
theta_v = np.linspace(min(theta_v), max(theta_v), 1024)
pr, theta = np.meshgrid(r_v, theta_v)
I = I.reshape(len(r_v), len(theta_v)).T

# I+=np.hstack((I[:,512:], I[:,0:512]))

px.imshow(I, origin='lower')

fft = np.fft.fft(I, axis=1)
fft[:, -1:] = 0
# px.imshow(np.abs(fft), origin='lower')
recovered = np.fft.ifft(fft, axis=1)
fig = px.imshow(np.abs(recovered), origin='lower', x=theta_v, y=r_v, aspect='auto')
tot = np.abs(fft[:, 0])

argmins = scipy.signal.argrelmin(recovered, axis=1)
r_min, theta_min = r_v[argmins[0]], theta_v[argmins[1]]

r_min, theta_min = r_min[(r_min > 0.15) & (r_min < 0.24)], theta_min[(r_min > 0.15) & (r_min < 0.24)]

fig.add_scatter(x=theta_min, y=r_min, mode='markers', marker=dict(size=5, color='red'), name='minima')
fig.show()

labels = DBSCAN(eps=0.15, min_samples=5).fit_predict(np.vstack((r_min * 15, theta_min)).T)

px.scatter(x=theta_min, y=r_min, color=labels).show()

theta_min = theta_min[labels == 0]
r_min = r_min[labels == 0]

# r_min=r_min[(theta_min>.8) & (theta_min<2)]
# theta_min=theta_min[(theta_min>0.8)&(theta_min<2)]
df = pd.DataFrame({'r': r_min, 'theta': theta_min})
df = df.groupby(['r']).mean().reset_index()
r_min = df['r']
theta_min = df['theta']

E = r_min ** 2 / 2

theta_smoothed = scipy.interpolate.make_smoothing_spline(E, theta_min, lam=1e-7)(E)

group_delay = np.gradient(theta_smoothed, E) * 2

px.line(x=E * 1.25, y=group_delay, markers=True).show()

tot = scipy.interpolate.make_smoothing_spline(r_v ** 2 / 2, tot, lam=1e-7)(E)
theory_df = pd.DataFrame({
    'pr': r_min,
    'E': E * 1.25,
    'theta': theta_smoothed,
    'group_delay': group_delay,
    'signal': tot / np.max(tot),
})
theory_df = theory_df[::6]



In [ ]:
fname = r"J:\ctgroup\Edward\DATA\VMI\20220613\xe005_e_calibrated.h5"
base_data = pd.read_hdf(fname, key="data")
data = base_data.query("sqrt(px**2+py**2+pz**2)<0.4").reset_index(drop=True)
data = data.query("abs(py)<0.5").reset_index(drop=True)
data = data.query("pz>0")
#
data_sym = data.copy()
data_sym['px'] = -data_sym['px']
data_sym['py'] = -data_sym['py']
data_sym['pz'] = -data_sym['pz']
data = pd.concat([data, data_sym])
del data_sym
data['pr'] = np.sqrt(data['px'] ** 2 + data['py'] ** 2 + data['pz'] ** 2)
data['phi'] = np.arctan2(data['px'], data['pz'])
data['theta'] = np.arccos(data['px'] / data['pr'])
data['E'] = data.pr ** 2 / 2

In [ ]:
h2d, re, pe = np.histogram2d(data['pr'], data['phi'], bins=1024)
re = 1 / 2 * (re[:-1] + re[1:])
pe = 1 / 2 * (pe[:-1] + pe[1:])
rr, pp = np.meshgrid(re, pe)
px.imshow(h2d, x=pe, y=re, origin='lower', aspect='auto')

h2d *= bump((rr.T - 0.25) / 0.1)

fft = np.fft.fft(h2d, axis=1)
# fft[:,9:]=0
# recovered=np.fft.ifft(fft, axis=1)

recovered = scipy.ndimage.gaussian_filter(h2d, sigma=15)

argmins1 = scipy.signal.argrelmin((recovered), axis=0)
argmins2 = scipy.signal.argrelmin((recovered), axis=1)
tot = np.abs(fft[:, 0])
# r_min, theta_min = np.hstack((re[argmins1[0]], re[argmins2[0]])), np.hstack((pe[argmins1[1]], pe[argmins2[1]]))
r_min, theta_min = re[argmins2[0]], pe[argmins2[1]]
mask = np.argwhere((theta_min > 0.15))
r_min, theta_min = r_min[mask], theta_min[mask]

r_min, theta_min = r_min[(r_min > 0.17) & (r_min < 0.32)], theta_min[(r_min > 0.17) & (r_min < 0.32)]
labels = DBSCAN(eps=0.15, min_samples=5).fit_predict(np.vstack((r_min * 15, theta_min)).T)

fig = px.imshow(np.abs(recovered), origin='lower', x=pe, y=re, aspect='auto',
                color_continuous_scale=px.colors.sequential.Inferno, labels=dict(color=r"S(p,ϕ)"))
fig.add_scatter(x=theta_min, y=r_min, mode='markers', marker=dict(size=5, color='red', symbol="circle"),
                name='Local Angular Minima')
fig.show()
px.scatter(x=theta_min, y=r_min, color=labels).show()
r_min = r_min[(labels == 0) | (labels == 4)]
theta_min = theta_min[(labels == 0) | (labels == 4)]
labels = labels[(labels == 0) | (labels == 4)]
mask = (r_min < 0.27) & (theta_min > 0.3) & (theta_min < 1.4)  #| (labels==4)
r_min, theta_min = r_min[mask], theta_min[mask]
df = pd.DataFrame({'r': r_min, 'theta': theta_min})
df = df.groupby(['theta']).min().reset_index()
r_min = df['r']
theta_min = df['theta']
E = r_min ** 2 / 2
th = np.linspace(min(theta_min), max(theta_min), 300)
r_smoothed = scipy.interpolate.make_smoothing_spline(theta_min, r_min, lam=1e-3)(th)
crs = np.max(h2d, axis=1) - np.min(h2d, axis=1)
cr = crs[np.intersect1d(pe, theta_min, return_indices=True)[1]]

E_min = r_min ** 2 / 2
polyfit = np.polyfit(E_min, theta_min, 2, w=cr ** 1)
r_smoothed = np.linspace(min(r_min), max(r_min), 50)
E = r_smoothed ** 2 / 2
theta_smoothed = np.poly1d(polyfit)(E)

fig.add_scatter(x=theta_smoothed, y=r_smoothed, mode='lines', line_color="white", line_dash="dot",
                name='Extracted Minimum')
fig.update_layout(
        height=400, width=600,
        legend=dict(
                orientation="v",
                yanchor="bottom",
                y=0,
                xanchor="left",
                x=0,
                font_color="white",
                bgcolor="black"
        )

)
fig.layout.template = "seaborn+presentation"
fig.update_xaxes(
        title_text=r"$\phi$ (rad)$",
        ticks="outside",
        ticklen=6,
        range=[-np.pi, np.pi]
)
fig.update_yaxes(
        title_text="$p_r$ (amu)$",
        ticks="outside",
        ticklen=6,
)
# theta_smoothed=th
fig.show()
fig.write_image(
        r"C:\Users\mcman\DataspellProjects\vmi-analysis\Minimum Extraction.png",
        scale=5
)

group_delay = np.gradient(theta_smoothed, E) * 2
px.line(x=E, y=theta_smoothed, markers=True).show()
px.line(x=E, y=group_delay, markers=True).show()

tot = scipy.interpolate.make_smoothing_spline(re ** 2 / 2, tot, lam=1e-8)(E)
experiment_df = pd.DataFrame({
    'pr': r_smoothed,
    'E': E,
    'theta': theta_smoothed - 2.62941 - np.pi,
    'group_delay': group_delay,
    'signal': tot / np.max(tot),
})

In [ ]:
theory_vortex_file = r"D:\DATA\Xe\Xe-vortex-diffpha-tdse.dat"
lines = []
for line in open(theory_vortex_file):
    line = line.strip()
    line = line.split(" ")
    line = [float(l) for l in line if l]
    lines.append(line)

p, c6, c4, c8, c7, phi, *_, dphide = map(np.asarray, zip(*lines))

theory_vortex = pd.DataFrame({
    "pr": p * np.sqrt(1.25),
    "E": p ** 2 / 2 * 1.25,
    "c6": c6,
    "c4": c4,
    "c8": c8,
    "c7": c7,
    "phi": phi,
    "dphide": dphide,
    "group_delay": dphide,
})

theory_vortex["signal"] = c6 + c7 + c8 + c4

theory_vortex_filt = theory_vortex.query("0.3<E*27.211386246<1.1")

px.line(x=theory_vortex['E'], y=theory_vortex['c6']).add_scatter(
        x=theory_vortex['E'], y=theory_vortex['c4']).add_scatter(
        x=theory_vortex['E'], y=theory_vortex['c8']).add_scatter(
        x=theory_vortex['E'], y=theory_vortex['c7']
).show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
        rows=3, cols=1,
        vertical_spacing=0.05,
        shared_xaxes=True,
        row_heights=[0.1, 0.4, 0.5],
).update_layout(height=800, width=600, margin=dict(t=0, r=0, b=0, l=0))
fig.layout.template = 'seaborn+presentation'

fig.add_trace(
        go.Scatter(
                x=theory_df['E'] * 27.211386246,
                y=theory_df['signal'],
                name='Signal',
                showlegend=False,
                line_color='blue',
                fill='tozeroy',
        )
)

fig.add_trace(
        go.Scatter(
                x=experiment_df['E'] * 27.211386246,
                y=experiment_df['signal'],
                name='Signal',
                showlegend=False,
                line_color='red',
                fill='tozeroy',
        )
)

# fig.add_trace(
#         go.Scatter(
#                 x=theory_vortex_filt['E']*27.211386246,
#                 y=theory_vortex_filt['signal']/max(theory_vortex_filt.signal),
#                 name='Signal',
#                 showlegend=False,
#                 line_color='darkblue',
#                 line_dash="dash",
#                 # fill='tozeroy',
#         )
# )

fig.update_yaxes(
        title_text="Signal",
        row=1, col=1,
        range=[0.4, 1.1],
        # minor_showgrid=True,
        ticks="outside",
        ticklen=6,
        tickvals=[0.5, 1],
)

# fig.add_trace(
#     go.Scatter(
#             x=theory_vortex_filt['E']*27.211386246,
#             y=theory_vortex_filt['group_delay']*0.02419,
#             mode='lines',
#             name='TDSE (Direct Phase)',
#             line_color='darkblue',
#             line_dash='dash',
#     ),
#     row=3, col=1
# )

fig.add_trace(
        go.Scatter(
                x=theory_df['E'] * 27.211386246,
                y=theory_df['group_delay'] * 0.02419,
                name='TDSE', mode='lines+markers',
                line=dict(width=2, color="blue"),
                marker=dict(size=5, color="blue", symbol="cross"),
        ),
        row=3, col=1
)

fig.add_trace(
        go.Scatter(
                x=experiment_df['E'] * 27.211386246,
                y=experiment_df['group_delay'] * 0.02419,
                mode='lines+markers', name='Experiment',
                line=dict(width=2, color="red"),
                marker=dict(size=5, color="red", symbol="circle"),
        ),
        row=3, col=1
)

fig.add_hline(
        y=2.669,
        line_color='black',
        line_width=1,
        opacity=1,
        line_dash='dashdot',
        annotation_text="Laser Period",
        annotation_position="bottom left",
        # annotation_font_size=12,
        # annotation_font_weight='bold',
        row=3, col=1,
)

fig.update_yaxes(
        title_text="Group Delay (fs)",
        row=3, col=1,
        range=[0, 4],
        minor_showgrid=True,
        ticks="outside",
        ticklen=6,
        tickvals=[0, 1, 2, 3, 4]
)
#
# fig.add_trace(
#     go.Scatter(
#             x=theory_vortex_filt['E']*27.211386246,
#             y=(theory_vortex_filt['phi']+np.pi)/theory_vortex_filt['E']*0.02419,
#             mode='lines',
#             name='TDSE Phase Delay',
#             showlegend=False,
#             line_color='darkblue',
#             line_dash='dash',
#     ),
#     row=2, col=1
# )

fig.add_trace(
        go.Scatter(
                x=theory_df['E'] * 27.211386246,
                y=2 * (theory_df['theta'] + (np.pi)) / theory_df['E'] * 0.02419,
                showlegend=False, name='TDSE', mode='lines+markers',
                line=dict(width=2, color="blue"),
                marker=dict(size=5, color="blue", symbol="cross"),
        ),
        row=2, col=1
)

fig.add_trace(
        go.Scatter(
                x=experiment_df['E'] * 27.211386246,
                y=2 * (experiment_df['theta'] + 2 * np.pi) / experiment_df['E'] * 0.02419,
                showlegend=False,
                mode='lines+markers', name='Experiment',
                line=dict(width=2, color="red"),
                marker=dict(size=5, color="red", symbol="circle"),
        ),
        row=2, col=1
)

fig.update_xaxes(
        range=[0.3, 1.1],
        minor_showgrid=True,
        ticks="outside",
        ticklen=6,

)

fig.update_xaxes(
        title_text="Energy (eV)",
        row=3, col=1,

)

fig.update_yaxes(
        title_text="Phase Delay (fs)",
        row=2, col=1, range=[2, 3],
        minor_showgrid=True,
        ticks="outside",
        ticklen=6,
)
fig.update_layout(
        legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
        )
)

fig.add_annotation(
        text="(a)",
        xref="paper",
        yref="y domain",
        x=0.025,
        y=0.95,
        showarrow=False,
)
fig.add_annotation(
        text="(b)",
        xref="paper",
        yref="y2 domain",
        x=0.025,
        y=0.975,
        showarrow=False,
)
fig.add_annotation(
        text="(c)",
        xref="paper",
        yref="y3 domain",
        x=0.025,
        y=0.975,
        showarrow=False,
)
fig.add_annotation(
        text=r"$\tau_{p}=\hbar \frac{\varphi}{E}$",
        xref="x2",
        yref="y2",
        x=1,
        y=2.9,
        showarrow=False,
)
fig.add_annotation(
        text=r"$\tau_{g}=\hbar \frac{d\varphi}{dE}$",
        xref="x3",
        yref="y3",
        x=1,
        y=0.5,
        showarrow=False,
)

fig.show()
fig.write_image(
        r"C:\Users\mcman\DataspellProjects\vmi-analysis\Xenon Delay Figure.png",
        scale=5
)


In [ ]:
fig2 = make_subplots(rows=1, cols=3, specs=[[{"l": 0, "r": -0.02}, {"l": -0.02}, {"l": 0.02}]],
                     column_widths=[0.3, 0.3, 0.5]
                     ).update_layout(height=375, width=1200, margin=dict(l=0, r=0, t=25, b=0), )
fig2.layout.template = "seaborn+presentation"

hist, xe, ye = np.histogram2d(
        data.pr * np.cos(data.phi - 2.62941),
        data.pr * np.sin(data.phi - 2.62941),
        bins=512, range=[[-0.35, 0.35], [-0.35, 0.35]], density=True,
)
xe = 1 / 2 * (xe[:-1] + xe[1:])
ye = 1 / 2 * (ye[:-1] + ye[1:])
xx, yy = np.meshgrid(xe, ye)
rr, tt = np.sqrt(xx ** 2 + yy ** 2), np.arctan2(yy, xx)
mask = bump((rr - 0.25) / 0.1)
hist *= mask
hist /= np.sum(hist)
dog_hist = skimage.filters.difference_of_gaussians(hist, 15, 20)
fig2.add_trace(go.Heatmap(
        x=xe,
        y=ye,
        z=hist.T,
        colorscale=px.colors.sequential.Inferno,
        showscale=False,
), row=1, col=1
)
fig2.add_trace(go.Scatter(
        x=experiment_df['pr'] * np.cos(experiment_df['theta']),
        y=experiment_df['pr'] * np.sin(experiment_df['theta']),
        mode='lines',
        line_color='white',
        line_dash='dashdot',
        showlegend=False,
))

pr = d[:, 0]
theta = d[:, 1] % (2 * np.pi) - np.pi
I = d[:, 4]

grid_size = (len(set(pr)), len(set(theta)))
pr = pr.reshape(grid_size)
r_v = np.asarray(sorted(np.unique(pr)))
theta = theta.reshape(grid_size)
theta_v = np.asarray(sorted(np.unique(theta)))
I = I.reshape(grid_size)

xv = np.linspace(-0.35, 0.35, 512)
yv = np.linspace(-0.35, 0.35, 512)
xx, yy = np.meshgrid(xv, yv)
rr, tt = np.sqrt(xx ** 2 + yy ** 2), np.arctan2(xx, yy) % (2 * np.pi)
rr_corr = rr / np.sqrt(1.25)

I_cart = scipy.ndimage.map_coordinates(
        I, [rr_corr.flatten() / np.max(r_v) * len(r_v), tt.flatten() * 180 / np.pi],
).reshape((512, 512)).T * mask
fig2.add_trace(go.Heatmap(
        x=xv, y=yv,
        z=I_cart,
        colorscale=px.colors.sequential.Inferno,
        showscale=False,

), row=1, col=2,
)
fig2.add_trace(go.Scatter(
        x=theory_df.pr * np.cos(theory_df.theta + np.pi) * np.sqrt(1.25),
        y=theory_df.pr * np.sin(theory_df.theta + np.pi) * np.sqrt(1.25),
        mode='lines',
        line_color='white',
        line_dash='dashdot',
        showlegend=False,
), row=1, col=2, )

fig2.update_xaxes(
        title='Major Axis (amu)',
        ticks='outside',
        range=[-0.35, 0.35],
        ticklen=6
)
fig2.update_yaxes(
        title='Minor Axis (amu)',
        ticks='outside',
        ticklen=6,
        scaleanchor='x',
        # range=[-0.35,0.35],
        row=1,
        col=1,
)
fig2.update_yaxes(
        row=1, col=2,
        ticks='outside',
        ticklen=6,
        showticklabels=False,
        scaleanchor='x',
)
fig2.update_xaxes(
        row=1, col=2,
        scaleanchor='x',
)
fig2.add_annotation(
        text="(a)",
        xref='x domain',
        yref='y domain',
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color='white',
)
fig2.add_annotation(
        text="(b)",
        xref='x2 domain',
        yref='y2 domain',
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color='white',
)
fig2.add_annotation(
        text="(c)",
        xref='x3 domain',
        yref='y3 domain',
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color='black',
)

line_params = [
    ("c4", 4, "blue"),
    ("c6", 6, "red"),
    ("c7", 7, "orange"),
    ("c8", 8, "green"),
]
cnorm = theory_vortex_filt.c6.max()
fig2.add_traces(
        [go.Scatter(
                x=theory_vortex_filt.E * 27.211386246,
                y=theory_vortex_filt[c[0]] / cnorm,
                name=f"l={c[1]}",
                line_color=c[2],
                line_width=2,
                marker=dict(size=5, color=c[2], symbol="circle"),
                mode="lines+markers",
        ) for c in line_params],
        rows=1, cols=3,
)
fig2.update_xaxes(
        title_text="Energy (eV)",
        row=1, col=3,
        ticks="outside",
        ticklen=6,
        range=[0.3, 1.1],
        minor_showgrid=True,
)

fig2.update_yaxes(
        title_text="Population (AU)",
        row=1, col=3,
        ticks="outside",
        ticklen=6,
        range=[-0.1, 1.1],
        minor_showgrid=True,
)

fig2.update_layout(
        legend=dict(
                yanchor="top",
                xanchor="right",
                x=1,
                y=1,
                font_size=14,
                # bordercolor="lightgrey",
                # borderwidth=1,
                orientation='v'
        )
)


def plot_ellipse(col):
    theta = np.linspace(0, 2 * np.pi, 100)
    r = 0.06
    x0 = 0.24
    y0 = -0.26
    x = r * np.cos(theta) + x0
    y = r * np.sin(theta) * 0.6 + y0

    fig2.add_scatter(
            x=x, y=y, row=1, col=col, line_color="red",
            showlegend=False,
    )
    fig2.add_scatter(
            x=[x0], y=[y0 - r * 0.6], row=1, col=col, mode="markers",
            marker=dict(color="red", symbol="triangle-right", size=12),
            showlegend=False,
    )
    fig2.add_scatter(
            x=[x0], y=[y0 + r * 0.6], row=1, col=col, mode="markers",
            marker=dict(color="red", symbol="triangle-left", size=12),
            showlegend=False,
    )


plot_ellipse(1)
plot_ellipse(2)

fig2.show()
fig2.write_image(
        r"C:\Users\mcman\DataspellProjects\vmi-analysis\Xenon Vortex Figure.png",
        scale=5
)

